In [ ]:
pip install BeautifulSoup4

In [ ]:
# pip install quandl

In [ ]:
pip install pandas_datareader

In [ ]:
pip install pandas

In [ ]:
import os 
dir_name= os.getcwd()+'/data/'

In [ ]:
# get data input for nasdaq
import pandas as pd

hkex_files=os.path.join(dir_name,'stock_ticker_datasets/hkex.csv')


hkex=pd.read_csv(hkex_files) 


hkex['Ticker']=hkex['Ticker'].astype(str)
hkex_input=hkex['Ticker']


n = 400  #chunk row size
hkex_df = [hkex_input[i:i+n] for i in range(0,hkex_input.shape[0],n)]
hkex_del=[hkex_input[i:i+n] for i in range(9999,hkex_input.shape[0],n)]
hkex.set_index("Ticker" , inplace=True)
print(hkex.head())


In [ ]:
from bs4 import BeautifulSoup
from urllib.request import Request
from urllib.request import urlopen
import csv
import datetime, time


prefix_url='http://www.aastocks.com/en/stocks/analysis/stock-aafn/'
postfix_url='/0/all/1'

def get_news_aastock(ticker):

    try:
        fill_ticker=ticker.zfill(5)
        url=prefix_url+fill_ticker+postfix_url
        print(url)
        req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
        resp = urlopen(req) 
        html = BeautifulSoup(resp, features="lxml") 
        dates=html.findAll("div", {"class": "inline_block"})
        news=html.findAll("div", {"class": "newshead4"})
        idx=0
        
        path=os.path.join(dir_name,'data-news/data-aastock/'+'data-'+ticker+'-aastock.csv')
        with open(path,'w') as f:
            writer = csv.writer(f)
            for i in dates:
                if "/" in str(i.get_text()):
                    date=str(i.get_text())

                    if "Release Time" in date:
                        date=date[13:23]
                    else:
                        date=str(date[:10])
                        text=news[idx].get_text()
                        date_time_obj = datetime.datetime.strptime(date, '%Y/%m/%d')
                                                         .strftime('%Y-%m-%d')
                        print(ticker)
                        print(date_time_obj)
                        if(datetime.datetime.now()-date_time_obj).days<=day:
                            category=hkex.loc[ticker]['Category']
                            print(text)
                            writer.writerow([date_time_obj,text])
                        idx+=1
    except:
        print('error')
        pass
    



In [ ]:
for tickers in hkex_df:
    for ticker in tickers:
        print(ticker)
        get_news_aastock(ticker)
            


In [ ]:
import os 
import datetime, time
# get agg news
df_hkex= pd.DataFrame()

for tickers in hkex_df:
     for ticker in tickers:
            try:
                f_ticker=ticker.zfill(5)
                l_ticker=ticker.zfill(4)
                news_path=os.path.join(dir_name,'data-news/data-aastock/'+'data-'+f_ticker+'-aastock.csv')
                dates_path=os.path.join(dir_name,'stock_label/hk_stock_label/'+'data-'+l_ticker+'-label.csv')
                
                df_news = pd.read_csv( df_news,names=['dates','news'],index_col='dates')
                df_dates=pd.read_csv(dates_path,names=['dates','label'],index_col='dates')
                merge=pd.merge(df_news,df_dates, how='inner', left_index=True, right_index=True)
                df_hkex=df_hkex.append(merge)
                print(df_hkex)
                
            except:
                print('no such file exist')
                pass

df_hkex.to_csv('hkex_labelled_news.csv')



In [ ]:
import os 
import datetime, time
# get agg news
df_hkex= pd.DataFrame()

for tickers in hkex_df:
     for ticker in tickers:
            try:
#                 f_ticker=ticker.zfill(5)
                news_path=os.path.join(dir_name,'data-news/data-aastock/'+'data-'+ticker+'-aastock.csv')
                print(news_path)
                df_news = pd.read_csv(news_path,names=['dates','news'])
                print(news_path)
                category=hkex.loc[ticker]['Category']
                Ticker=[ticker for i in range(len(df_news))]
                Category=[category for i in range(len(df_news))]
                df_news['ticker']=Ticker
                df_news['category']=Category
                df_hkex=df_hkex.append(df_news)
                print(df_hkex)
                
            except:
                print('no such file exist')
                pass
hkex_path=os.path.join(dir_name,'train-data/hkex/hkex_no_labelled_news.csv')
df_hkex.to_csv(hkex_path)